In [2]:
"""
!unzip "/content/test.zip"
!unzip "/content/train.zip"
!unzip "/content/val.zip"
"""

'\n!unzip "/content/test.zip"\n!unzip "/content/train.zip"\n!unzip "/content/val.zip"\n'

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import cv2
import glob

from keras.models import Model
from keras.layers import Input, Conv2D, Activation, add, AveragePooling2D, Flatten, Dense
from keras.backend import sigmoid

import torch
import torchvision
import torch.nn as nn
from torch.nn import AvgPool2d, Linear, Flatten, Sigmoid
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.preprocessing import LabelEncoder


from PIL import Image
import torch.nn.functional as F

PYTORCH CNN

In [5]:
def Swish(x):
  return x * torch.sigmoid(x)

class CNN(nn.Module): # derin ogrenme modeli
  def __init__(self, number_of_classes = 9):
    super(CNN, self).__init__()

    # Layer 1 --> Convolutional Layer 1
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1)

    # Layer 2 --> Convolutional Layer 2
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)

    # Layer 3 --> Convolutional Layer 3

    self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)

    # Layer 4 --> Convolutional Layer 4
    self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)

    # Layer 5 --> Convolutional Layer 5
    self.conv5 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)

    # Layer 6 --> Convolutional Layer 6
    self.conv6 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)

    # Layer 7 --> Convolutional Layer 7
    self.conv7 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)

    # Layer 8 --> Convolutional Layer 8
    self.conv8 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)

    # Layer 9 --> Convolutional Layer 9
    self.conv9 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)

    # Layer 10 --> Pooling Layer
    self.average_pooling = AvgPool2d(2)

    # Layer 11 --> Flatten Layer
    self.flatten = Flatten()

    # Layer 12 --> Fully Connected Layer 1
    self.fully_connected1 = Linear(in_features = (512 * 7 * 7), out_features = 1024)

    # Layer 13 --> Fully Connected Layer 2
    self.fully_connected2 = Linear(in_features = 1024, out_features = 512)

    # Layer 14 --> Fully Connected Layer 3
    self.fully_connected3 = Linear(in_features = 512, out_features = 9)


  def forward(self, x):
    c1 = Swish(self.conv1(x))
    c2 = Swish(self.conv2(c1))
    c3 = Swish(self.conv3(c2))
    c4 = Swish(self.conv4(c3))
    c5 = Swish(self.conv5(c4))
    c6 = Swish(self.conv6(c5))
    c7 = Swish(self.conv7(c6))
    c8 = Swish(self.conv8(c7))
    c9 = Swish(self.conv9(c8))

    out = self.average_pooling(c9)

    out = self.flatten(out)

    #if self.fully_connected1 is None:
     # input_features = out.size(1) # out5 --> [batch_size, channels, height, width] biz de channels sayisini aldik
     # self.fully_connected1 = Linear(in_features = input_features, out_features = 2048)
     # self.fully_connected2 = Linear(in_features = 2048, out_features = 1024)
     # self.fully_connected3 = Linear(in_features = 1024, out_features = 9)

    out = self.fully_connected1(out)
    out = self.fully_connected2(out)
    out = self.fully_connected3(out)

    return out

model = CNN() # model olusturuldu

In [6]:
path_train_images = glob.glob("/content/train/**/*.jpg", recursive = True)
path_validation_images = glob.glob("/content/val/**/*.jpg", recursive = True)
path_test_images = glob.glob("/content/test/**/*.jpg", recursive = True)

LOAD AND NORMALIZE DATA

In [7]:
def Custom_Dataset(image_paths, transform = None):
  label_encoder = LabelEncoder()

  labels = []
  images = []

  for i, image_path in enumerate(image_paths):
    image = cv2.imread(image_path)
    label = image_paths[i].split('/')[-2]  # split hatasını düzelttim


    if transform:
      image = transform(Image.fromarray(image))  #

    images.append(torch.as_tensor(image, dtype=torch.float32).clone().detach())
    labels.append(label)

  encoded_labels = label_encoder.fit_transform(labels)

  return TensorDataset(torch.stack(images), torch.tensor(encoded_labels, dtype=torch.long))

In [8]:
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

train_dataset = Custom_Dataset(image_paths = path_train_images, transform = transform)
validation_dataset = Custom_Dataset(image_paths = path_validation_images, transform = transform)
test_dataset = Custom_Dataset(image_paths = path_test_images, transform = transform)

train_dataloader = DataLoader(train_dataset, batch_size = 16, shuffle = True)
validation_dataloader = DataLoader(validation_dataset, batch_size = 16, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 16, shuffle = True)

In [9]:
def to_numpy(tensor):
    with torch.no_grad():
        return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy().astype(np.float32)

def validation_loop(validation_dataloader, model, loss_fn):
    model.eval()  # Modeli değerlendirme moduna geçir

    size = len(validation_dataloader.dataset)
    num_batches = len(validation_dataloader)
    total_loss, correct = 0, 0

    with torch.no_grad():
        for validation_data, validation_labels in validation_dataloader:
            pred = model(validation_data)
            total_loss += loss_fn(pred, validation_labels).item()
            correct += ((to_numpy(pred.argmax(1)).detach()) == to_numpy(validation_labels.detach())).astype(np.float).sum()

    total_loss /= num_batches
    correct /= size

    print("Validation Error:\n", "Accuracy:", round((100 * correct), 4), " | Average Loss:", round(total_loss, 4), "\n")

In [10]:
from torch.autograd import detect_anomaly
def train_loop(dataloader, model, loss_function, optimizer, validation_dataloader, scheduler, accumulation_steps):
    size = len(dataloader.dataset)
    model.train()

    total_loss, correct = 0, 0
    toplam_kayip = 0

    for batch, (image, label) in enumerate(dataloader):
        pred = model(image)
        loss = loss_function(pred, label)

        # Gradient biriktirme
        loss.backward()
        toplam_kayip += loss.item()

        if (batch + 1) % accumulation_steps == 0:
            # Gradient biriktirme adımında bir güncelleme yap
            optimizer.step()
            optimizer.zero_grad()

            # Güncelleme sonrasında toplam kaybı yazdır
            current = (batch + 1) * len(image)
            print("| Batch:", current, "| Total Loss:", toplam_kayip, "\n")
            toplam_kayip = 0

    # Gradient biriktirme adımları bittiğinde son güncellemeyi yap
    if toplam_kayip > 0:
        optimizer.step()
        optimizer.zero_grad()

    # Validation için modeli değerlendir
    validation_loop(validation_dataloader, model, loss_function)
    model.train()  # Eğitim moduna geri geç

    # Öğrenme oranını güncelle
    scheduler.step()

In [11]:
def test_loop(dataloader, model, loss_fn):

    model.eval() # Test yaptigimiz icin eval moddayiz
    size = len(dataloader.dataset)
    num_batches = len(dataloader) # toplam batch sayisi
    total_loss, correct = 0, 0

    with torch.no_grad(): # gradyan hesaplaması yapmamak icin. degerlendirmede parametreleri guncellemeyiz
        for image, label in dataloader:
            pred = model(image) # tahmin
            total_loss += loss_fn(pred, label).item() # toplam kayip
            correct += ((to_numpy(pred.argmax(1))).detach() == to_numpy(label.detach())).astype(np.float).sum()  # dogru tahmin

    # ortalama kayip ve dogruluk orani
    total_loss /= num_batches
    correct /= size

    print("Test Error:\n", "Accuracy:", round((100 * correct),4)," | Average Loss:",round(total_loss,4), "\n")

In [ ]:
number_of_epochs = 60
learning_rate = 1e-3
batch_size = 4
accumulation_steps = 4  # Her biriktirme adımında bir güncelleme yapılacak
# optimizer --> Adam

loss_function = nn.CrossEntropyLoss() #siniflandirma icin kul. yaygin loss func.

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)  # Adam optimizer
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 40], gamma=0.3)  # belirli araliklarla lr degistirmek icin

epochs = number_of_epochs
for epoch in range(epochs):
    print(f"Epoch \n-------------------------------")
    train_loop(train_dataloader, model, loss_function, optimizer, validation_dataloader, scheduler,accumulation_steps)
    #test_loop(test_dataloader, model, loss_function)
print("Done!")

Epoch 
-------------------------------
